In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from icecream import ic
from sklearn.preprocessing import LabelEncoder
import time
from itertools import product
from icecream import ic


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

sales_train = pd.read_csv('./data/sales_train.csv')
test = pd.read_csv('./data/test.csv')   # (214200, 3)

# 计算每个商品每个月的销售量，假如某个商品在某个月没有数据，则填充0（即这个月的销售量为0）
sales_by_item_id = sales_train.pivot_table(index=['item_id'], values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
sales_by_item_id.columns = sales_by_item_id.columns.droplevel().map(str)   # 去掉第一层索引
sales_by_item_id.columns.values[0] = 'item_id'
sales_by_item_id = sales_by_item_id.rename_axis(None, axis=1)

# 获取最近6个月销售量为0的数据
six_zero = sales_by_item_id[(sales_by_item_id['28'] == 0) & (sales_by_item_id['29'] == 0) & (sales_by_item_id['30'] == 0) & (sales_by_item_id['31'] == 0) & (sales_by_item_id['32'] == 0) & (sales_by_item_id['33'] == 0)]
six_zero_item_id = list(six_zero['item_id'].values)   # item_id列表
# test.loc[test.item_id.isin(six_zero_item_id), 'item_cnt_month'] = 0  # 将test数据中（最近六个月销量为0）的数据月销量设为0，有7812个

# 计算每个商店每个月的销量
sales_by_shop_id = sales_train.pivot_table(index=['shop_id'], values=['item_cnt_day'], aggfunc=np.sum, fill_value=0, columns='date_block_num').reset_index()
sales_by_shop_id.columns = sales_by_shop_id.columns.droplevel().map(str)    # 将两层column转化为一层column,保留下层column
sales_by_shop_id.columns.values[0] = 'shop_id'
sales_by_shop_id = sales_by_shop_id.rename_axis(None, axis=1)   # 将列方向的轴重命名为none

# zero = sales_train[sales_train.date_block_num==0]
# ic(zero.shop_id.unique(), len(zero.item_id.unique()), len(zero.shop_id.unique()), len(zero.shop_id.unique()) * len(zero.item_id.unique()))
# ic(sales_train.shop_id.unique(), len(sales_train.item_id.unique()), len(sales_train.shop_id.unique()), len(sales_train.shop_id.unique()) * len(sales_train.item_id.unique()))

"""组合date_block_num,shop_id,item_id(部分) 总量：10913850"""
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = sales_train[sales_train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix.sort_values(cols, inplace=True)  # 排序
sales_train['revenue'] = sales_train['item_price'] * sales_train['item_cnt_day']    # 某一天的销售额

# 分组
groupby = sales_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day': 'sum'}).reset_index()
groupby = groupby.rename(columns={'item_cnt_day': 'item_cnt_month'})
matrix = matrix.merge(groupby, on=['date_block_num','shop_id','item_id'], how='left')
matrix['item_cnt_month'] = matrix['item_cnt_month'].fillna(0).clip(0, 20)
matrix['item_cnt_month'] = matrix['item_cnt_month'].astype(np.float16)

# test数据
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

# 合并matrix,test
matrix = pd.concat([matrix, test[cols]], ignore_index=True, axis=0)
matrix['item_cnt_month'].fillna(0, inplace=True)

# 商品信息
items = pd.read_csv('./data/items.csv')
items = items[['item_id', 'item_category_id']]
matrix = pd.merge(left=matrix, right=items, on='item_id', how='left')  # 合并

# 商品类别
le = LabelEncoder()
categories = pd.read_csv('./data/item_categories.csv')
categories['split'] = categories['item_category_name'].str.split('-')
categories['type'] = categories['split'].map(lambda x:x[0].strip())
categories['subtype'] = categories['split'].map(lambda x:x[1].strip() if len(x)>1 else x[0].strip())
categories = categories[['item_category_id','type','subtype']]
categories['cat_type_code'] = le.fit_transform(categories['type'])
categories['cat_subtype_code'] = le.fit_transform(categories['subtype'])
matrix = pd.merge(left=matrix, right=categories[['item_category_id','cat_type_code','cat_subtype_code']], on='item_category_id', how='left')    # 合并

# 商店信息
shops = pd.read_csv('./data/shops.csv')
shops['split']=shops.shop_name.str.split(' ')
shops['shop_city'] = shops['split'].map(lambda x:x[0])
shops['shop_city_code'] = le.fit_transform(shops['shop_city'])

def st(name):
    if 'ТЦ' in name or 'ТРЦ' in name:
        shopt = 'ТЦ'
    elif 'ТК' in name:
        shopt = 'ТК'
    elif 'ТРК' in name:
        shopt = 'ТРК'
    elif 'МТРЦ' in name:
        shopt = 'МТРЦ'
    else:
        shopt = 'UNKNOWN'
    return shopt
shops['shop_type'] = shops['shop_name'].apply(st)

shops.loc[shops.shop_id == 21, 'shop_type'] = 'МТРЦ'   # 修正
shops['shop_type_code'] = le.fit_transform(shops['shop_type'])
matrix = pd.merge(left=matrix, right=shops[['shop_id','shop_city_code','shop_type_code']], on='shop_id', how='left')    # 合并
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['cat_type_code'] = matrix['cat_type_code'].astype(np.int8)
matrix['cat_subtype_code'] = matrix['cat_subtype_code'].astype(np.int8)
matrix['shop_city_code'] = matrix['shop_city_code'].astype(np.int8)
matrix['shop_type_code'] = matrix['shop_type_code'].astype(np.int8)


"""历史信息"""

def lag_features(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id',col+'_lag_'+str(i)]
        shifted['date_block_num'] = shifted['date_block_num'] + i
        df = pd.merge(left=df, right=shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

matrix = lag_features(matrix, [1,2,3,6,12], 'item_cnt_month')

# 月销量（所有商品）
group = matrix.groupby('date_block_num').agg({'item_cnt_month': 'mean'}).reset_index()
group.columns = ['date_block_num', 'date_avg_item_cnt']
matrix = pd.merge(left=matrix, right=group, on='date_block_num', how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_avg_item_cnt')
matrix.drop('date_avg_item_cnt', axis=1, inplace=True)

# 月销量（每一件商品）
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop('date_item_avg_item_cnt', axis=1, inplace=True)

# 月销量（每个商店 ）
group = matrix.groupby(['date_block_num','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop('date_shop_avg_item_cnt', axis=1, inplace=True)



# 月销量（每个类别）
group = matrix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_avg_item_cnt')
matrix.drop('date_cat_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品类别-商店）
group = matrix.groupby(['date_block_num','item_category_id','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_shop_avg_item_cnt')
matrix.drop('date_cat_shop_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品大类）
group = matrix.groupby(['date_block_num','cat_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_type_avg_item_cnt')
matrix.drop('date_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商品大类） ++++++++++++ 和 月销量（商品）是重复的，因为每一个商品，类别是确定的，大类也是确定的
group = matrix.groupby(['date_block_num', 'item_id', 'cat_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_type_avg_item_cnt')
matrix.drop('date_item_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商店城市）
group = matrix.groupby(['date_block_num','shop_city_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_city_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_city_avg_item_cnt')
matrix.drop('date_city_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店城市）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_city_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_city_avg_item_cnt')
matrix.drop('date_item_city_avg_item_cnt', axis=1, inplace=True)



# 趋势特征
group = sales_train.groupby('item_id').agg({'item_price': 'mean'})
group.columns = ['item_avg_item_price']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = sales_train.groupby(['date_block_num','item_id']).agg({'item_price': 'mean'})
group.columns = ['date_item_avg_item_price']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'date_item_avg_item_price')
for i in lags:
    matrix['delta_price_lag_'+str(i)] = (matrix['date_item_avg_item_price_lag_' + str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if pd.notnull(row['delta_price_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_price_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)

features_to_drop = ['item_avg_item_price','date_item_avg_item_price']
for i in lags:
    features_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    features_to_drop += ['delta_price_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

# 每个月的天数
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days)
matrix['days'] = matrix['days'].astype(np.int8)

# 开始销量
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

In [2]:
# 月销量（商店类型）
group = matrix.groupby(['date_block_num','shop_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shoptype_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shoptype_avg_item_cnt')
matrix.drop('date_shoptype_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店类型）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_shoptype_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_shoptype_avg_item_cnt')
matrix.drop('date_item_shoptype_avg_item_cnt', axis=1, inplace=True)

In [3]:
matrix.shape

(11128050, 74)

In [4]:
# 月销量（商店-商品）
group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shopitem_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shopitem_avg_item_cnt')
matrix.drop('date_shopitem_avg_item_cnt', axis=1, inplace=True)

In [33]:
# matrix_bak = matrix.copy()   # 已执行

(11128050, 79)

In [36]:
# 趋势特征
group = matrix.groupby('item_id').agg({'item_cnt_month': 'mean'})
group.columns = ['trend_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = matrix.groupby(['date_block_num','item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['trend_date_item_avg_cnt_month']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'trend_date_item_avg_cnt_month')
for i in lags:
    matrix['delta_cnt_month_lag_'+str(i)] = (matrix['trend_date_item_avg_cnt_month_lag_' + str(i)] - matrix['trend_item_avg_cnt_month']) / matrix['trend_item_avg_cnt_month']
    
def select_trend2(row):
    for i in lags:
        if pd.notnull(row['delta_cnt_month_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_cnt_month_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta_cnt_month_lag'] = matrix.apply(select_trend2, axis=1)

matrix['delta_cnt_month_lag'] = matrix['delta_cnt_month_lag'].astype(np.float16)

features_to_drop = ['trend_item_avg_cnt_month','trend_date_item_avg_cnt_month']
for i in lags:
    features_to_drop += ['trend_date_item_avg_cnt_month_lag_'+str(i)]
    features_to_drop += ['delta_cnt_month_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

In [37]:
matrix.shape

(11128050, 80)

In [40]:
# 趋势特征 +++++++++++++++++++++++++++++++++++++
group = matrix.groupby(['shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['qushi_shop_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['shop_id', 'item_id'], how='left')

group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['qushi_date_shop_item_avg_cnt_month']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')

In [41]:
matrix.shape

(11128050, 82)

In [42]:
# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'qushi_date_shop_item_avg_cnt_month')
for i in lags:
    matrix['delta2_cnt_month_lag_'+str(i)] = (matrix['qushi_date_shop_item_avg_cnt_month_lag_' + str(i)] - matrix['qushi_shop_item_avg_cnt_month']) / matrix['qushi_shop_item_avg_cnt_month']

In [43]:
def select_trend3(row):
    for i in lags:
        if pd.notnull(row['delta2_cnt_month_lag_'+str(i)]):  # 如果不是NaN
            return row['delta2_cnt_month_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta2_cnt_month_lag'] = matrix.apply(select_trend3, axis=1)

In [46]:
matrix['delta2_cnt_month_lag'] = matrix['delta2_cnt_month_lag'].astype(np.float16)

In [47]:
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128050 entries, 0 to 11128049
Data columns (total 97 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   date_block_num                             int8   
 1   shop_id                                    int8   
 2   item_id                                    int16  
 3   item_cnt_month                             float16
 4   item_category_id                           int8   
 5   cat_type_code                              int8   
 6   cat_subtype_code                           int8   
 7   shop_city_code                             int8   
 8   shop_type_code                             int8   
 9   item_cnt_month_lag_1                       float16
 10  item_cnt_month_lag_2                       float16
 11  item_cnt_month_lag_3                       float16
 12  item_cnt_month_lag_6                       float16
 13  item_cnt_month_lag_12                   

In [48]:
matrix.shape

(11128050, 97)

In [49]:
features_to_drop = ['qushi_shop_item_avg_cnt_month','qushi_date_shop_item_avg_cnt_month']
for i in lags:
    features_to_drop += ['delta2_cnt_month_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

In [50]:
matrix.shape

(11128050, 88)

In [51]:
# matrix_88_bak = matrix.copy()   # 已运行

In [52]:
# 因为有12个月的延迟特征（1，2，3，6，12）（1，2，3，4，5，6，12），所以需要删除前12月的数据
matrix = matrix[matrix['date_block_num'] > 11]

In [53]:
matrix.shape

(6639294, 88)

In [54]:
# 找到有NaN值的列，然后把那些列中的NaN值填充0
columns = matrix.columns
column_null = []
for i in columns:
    if len(matrix[matrix[i].isnull()]) > 0:
        column_null.append(i)

for i in column_null:
    matrix[i].fillna(0, inplace=True)

In [55]:
# matrix_before_model_bak = matrix.copy()   # 在建模前进行备份  已运行

In [56]:
"""建模"""

trainData = matrix[matrix['date_block_num'] < 33]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

validData = matrix[matrix['date_block_num'] == 33]
label_valid = validData['item_cnt_month']
X_valid = validData.drop('item_cnt_month', axis=1)

In [58]:
import lightgbm as lgb
train_data = lgb.Dataset(data=X_train, label=label_train)
valid_data = lgb.Dataset(data=X_valid, label=label_valid)
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 10000,
    'max_depth': 8,
    'num_leaves': 220,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.005,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data])

[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.326877 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13347
[LightGBM] [Info] Number of data points in the train set: 6186922, number of used features: 87
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Info] Start training from score 0.288852
[1]	training's rmse: 1.18576	valid_1's rmse: 1.13473
[2]	training's rmse: 1.18366	valid_1's rmse: 1.13328
[3]	training's rmse: 1.18134	valid_1's rmse: 1.13157
[4]	training's rmse: 1.17877	valid_1's rmse: 1.12963
[5]	training's rmse: 1.17627	valid_1's rmse: 1.12771
[6]	training's rmse: 1.1741	valid_1's rmse: 1.12621
[7]	training's rmse: 1.1716	valid_1's rmse: 1.12418
[8]	training's rmse: 1.1694	valid_1's rmse: 1

[140]	training's rmse: 0.959432	valid_1's rmse: 0.9742
[141]	training's rmse: 0.958348	valid_1's rmse: 0.97344
[142]	training's rmse: 0.957352	valid_1's rmse: 0.972875
[143]	training's rmse: 0.95665	valid_1's rmse: 0.972472
[144]	training's rmse: 0.955716	valid_1's rmse: 0.971956
[145]	training's rmse: 0.954476	valid_1's rmse: 0.971022
[146]	training's rmse: 0.953533	valid_1's rmse: 0.97044
[147]	training's rmse: 0.952659	valid_1's rmse: 0.969898
[148]	training's rmse: 0.951594	valid_1's rmse: 0.969165
[149]	training's rmse: 0.950734	valid_1's rmse: 0.968682
[150]	training's rmse: 0.949655	valid_1's rmse: 0.967956
[151]	training's rmse: 0.948598	valid_1's rmse: 0.96719
[152]	training's rmse: 0.947836	valid_1's rmse: 0.966701
[153]	training's rmse: 0.947009	valid_1's rmse: 0.966241
[154]	training's rmse: 0.946165	valid_1's rmse: 0.965749
[155]	training's rmse: 0.945538	valid_1's rmse: 0.965377
[156]	training's rmse: 0.944698	valid_1's rmse: 0.964855
[157]	training's rmse: 0.94368	valid_

[282]	training's rmse: 0.868304	valid_1's rmse: 0.922843
[283]	training's rmse: 0.867649	valid_1's rmse: 0.922427
[284]	training's rmse: 0.86726	valid_1's rmse: 0.922294
[285]	training's rmse: 0.866778	valid_1's rmse: 0.922061
[286]	training's rmse: 0.866208	valid_1's rmse: 0.921748
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[287]	training's rmse: 0.865756	valid_1's rmse: 0.921516
[288]	training's rmse: 0.865425	valid_1's rmse: 0.921358
[289]	training's rmse: 0.865063	valid_1's rmse: 0.921226
[290]	training's rmse: 0.864614	valid_1's rmse: 0.920999
[291]	training's rmse: 0.864302	valid_1's rmse: 0.920899
[292]	training's rmse: 0.86397	valid_1's rmse: 0.920786
[293]	training's rmse: 0.863664	valid_1's rmse: 0.920653
[294]	training's rmse: 0.863367	valid_1's rmse: 0.920544
[295]	training's rmse: 0.862846	valid_1's rmse: 0.920246
[296]	training's rmse: 0.862438	valid_1's rmse: 0.920073
[297]	training's rmse: 0.862138	valid_1's rmse: 0.919949
[298]	training'

[410]	training's rmse: 0.830552	valid_1's rmse: 0.908198
[411]	training's rmse: 0.830278	valid_1's rmse: 0.908054
[412]	training's rmse: 0.83011	valid_1's rmse: 0.908059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[413]	training's rmse: 0.829949	valid_1's rmse: 0.908054
[414]	training's rmse: 0.829705	valid_1's rmse: 0.908058
[415]	training's rmse: 0.829477	valid_1's rmse: 0.907962
[416]	training's rmse: 0.829307	valid_1's rmse: 0.907953
[417]	training's rmse: 0.828913	valid_1's rmse: 0.907746
[418]	training's rmse: 0.828689	valid_1's rmse: 0.907729
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[419]	training's rmse: 0.82844	valid_1's rmse: 0.907569
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[420]	training's rmse: 0.828213	valid_1's rmse: 0.907473
[421]	training's rmse: 0.827992	valid_1's rmse: 0.907414
[422]	training's rmse: 0.827763	valid_1's rmse: 0.907335
[423]	training's rmse: 0.827596	vali

[516]	training's rmse: 0.812319	valid_1's rmse: 0.90388
[517]	training's rmse: 0.812175	valid_1's rmse: 0.903764
[518]	training's rmse: 0.81207	valid_1's rmse: 0.903768
[519]	training's rmse: 0.811906	valid_1's rmse: 0.903684
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[520]	training's rmse: 0.811788	valid_1's rmse: 0.903681
[521]	training's rmse: 0.811635	valid_1's rmse: 0.903698
[522]	training's rmse: 0.811495	valid_1's rmse: 0.903699
[523]	training's rmse: 0.811394	valid_1's rmse: 0.90373
[524]	training's rmse: 0.811266	valid_1's rmse: 0.903735
[525]	training's rmse: 0.811047	valid_1's rmse: 0.90357
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[526]	training's rmse: 0.810937	valid_1's rmse: 0.903534
[527]	training's rmse: 0.810761	valid_1's rmse: 0.903504
[528]	training's rmse: 0.810576	valid_1's rmse: 0.90346
[529]	training's rmse: 0.810399	valid_1's rmse: 0.903346
[LightGBM] [Warning] No further splits with positive gain

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9727]	training's rmse: 0.681649	valid_1's rmse: 0.87687
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9728]	training's rmse: 0.681646	valid_1's rmse: 0.87687
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9729]	training's rmse: 0.681642	valid_1's rmse: 0.876872
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9730]	training's rmse: 0.681636	valid_1's rmse: 0.876871
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9731]	training's rmse: 0.681633	valid_1's rmse: 0.876873
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9732]	training's rmse: 0.68163	valid_1's rmse: 0.876874
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9733]	training's rmse: 0.681626	valid_1's rmse: 0.876875
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9790]	training's rmse: 0.681365	valid_1's rmse: 0.876814
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9791]	training's rmse: 0.68136	valid_1's rmse: 0.876813
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9792]	training's rmse: 0.681356	valid_1's rmse: 0.876813
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9793]	training's rmse: 0.681354	valid_1's rmse: 0.876813
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9794]	training's rmse: 0.68135	valid_1's rmse: 0.876807
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9795]	training's rmse: 0.681346	valid_1's rmse: 0.876798
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9796]	training's rmse: 0.681344	valid_1's rmse: 0.876799
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9854]	training's rmse: 0.681048	valid_1's rmse: 0.876705
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9855]	training's rmse: 0.681044	valid_1's rmse: 0.876707
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9856]	training's rmse: 0.681037	valid_1's rmse: 0.876708
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9857]	training's rmse: 0.681034	valid_1's rmse: 0.87671
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9858]	training's rmse: 0.681031	valid_1's rmse: 0.87671
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9859]	training's rmse: 0.681027	valid_1's rmse: 0.876708
[9860]	training's rmse: 0.681005	valid_1's rmse: 0.876693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9861]	training's rmse: 0.681001	valid_1's rmse: 0.876692
[LightGBM] [W

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9918]	training's rmse: 0.680712	valid_1's rmse: 0.876601
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9919]	training's rmse: 0.680702	valid_1's rmse: 0.876604
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9920]	training's rmse: 0.680699	valid_1's rmse: 0.876603
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9921]	training's rmse: 0.680697	valid_1's rmse: 0.876602
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9922]	training's rmse: 0.680694	valid_1's rmse: 0.8766
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9923]	training's rmse: 0.680691	valid_1's rmse: 0.876599
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9924]	training's rmse: 0.680687	valid_1's rmse: 0.876596
[LightGBM] [Warning] No further splits with positive gain, best gain: -

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9982]	training's rmse: 0.680437	valid_1's rmse: 0.876682
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9983]	training's rmse: 0.680435	valid_1's rmse: 0.876683
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9984]	training's rmse: 0.680431	valid_1's rmse: 0.876683
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9985]	training's rmse: 0.680428	valid_1's rmse: 0.876682
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9986]	training's rmse: 0.680425	valid_1's rmse: 0.876684
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9987]	training's rmse: 0.680422	valid_1's rmse: 0.876685
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9988]	training's rmse: 0.680417	valid_1's rmse: 0.87668
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [59]:
# test数据
testData = matrix[matrix['date_block_num'] == 34]
X_test = testData.drop('item_cnt_month', axis=1)

# 预测&生成文件
y_test = lgb_model.predict(X_test).clip(0, 20)
submission = pd.DataFrame({ 'ID': range(0, 214200), 'item_cnt_month': y_test})

test0 = test[test.item_id.isin(six_zero_item_id)]
ids = list(test0.ID.values)
submission.loc[submission.ID.isin(ids), 'item_cnt_month'] = 0.0
submission.to_csv('./submit/sub_876645.csv', index=False)

[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.230613 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12714
[LightGBM] [Info] Number of data points in the train set: 6186922, number of used features: 79
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Info] Start training from score 0.288852
[1]	training's rmse: 1.18582	valid_1's rmse: 1.13491
[2]	training's rmse: 1.18341	valid_1's rmse: 1.13319
[3]	training's rmse: 1.18116	valid_1's rmse: 1.13162
[4]	training's rmse: 1.17867	valid_1's rmse: 1.12981
[5]	training's rmse: 1.1761	valid_1's rmse: 1.1278
[6]	training's rmse: 1.17369	valid_1's rmse: 1.1261
[7]	training's rmse: 1.17176	valid_1's rmse: 1.1247
[8]	training's rmse: 1.16935	valid_1's rmse: 1.

[141]	training's rmse: 0.968215	valid_1's rmse: 0.983716
[142]	training's rmse: 0.967283	valid_1's rmse: 0.983201
[143]	training's rmse: 0.966334	valid_1's rmse: 0.982576
[144]	training's rmse: 0.965453	valid_1's rmse: 0.982042
[145]	training's rmse: 0.964553	valid_1's rmse: 0.981491
[146]	training's rmse: 0.963701	valid_1's rmse: 0.98092
[147]	training's rmse: 0.962844	valid_1's rmse: 0.980442
[148]	training's rmse: 0.962056	valid_1's rmse: 0.980013
[149]	training's rmse: 0.961198	valid_1's rmse: 0.979469
[150]	training's rmse: 0.96039	valid_1's rmse: 0.978955
[151]	training's rmse: 0.959571	valid_1's rmse: 0.978478
[152]	training's rmse: 0.958715	valid_1's rmse: 0.977921
[153]	training's rmse: 0.957904	valid_1's rmse: 0.977452
[154]	training's rmse: 0.956888	valid_1's rmse: 0.976747
[155]	training's rmse: 0.956029	valid_1's rmse: 0.976204
[156]	training's rmse: 0.955226	valid_1's rmse: 0.975707
[157]	training's rmse: 0.954278	valid_1's rmse: 0.975076
[158]	training's rmse: 0.953275	v

[278]	training's rmse: 0.885028	valid_1's rmse: 0.933885
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[279]	training's rmse: 0.884597	valid_1's rmse: 0.933631
[280]	training's rmse: 0.884251	valid_1's rmse: 0.933485
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[281]	training's rmse: 0.883919	valid_1's rmse: 0.933324
[282]	training's rmse: 0.883451	valid_1's rmse: 0.933052
[283]	training's rmse: 0.883144	valid_1's rmse: 0.932908
[284]	training's rmse: 0.882793	valid_1's rmse: 0.932715
[285]	training's rmse: 0.882357	valid_1's rmse: 0.932427
[286]	training's rmse: 0.88206	valid_1's rmse: 0.932286
[287]	training's rmse: 0.881758	valid_1's rmse: 0.932133
[288]	training's rmse: 0.881429	valid_1's rmse: 0.931972
[289]	training's rmse: 0.881096	valid_1's rmse: 0.931835
[290]	training's rmse: 0.880637	valid_1's rmse: 0.93153
[291]	training's rmse: 0.880321	valid_1's rmse: 0.931387
[292]	training's rmse: 0.879956	valid_1's rmse: 0.9311

[399]	training's rmse: 0.852468	valid_1's rmse: 0.918054
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[400]	training's rmse: 0.852248	valid_1's rmse: 0.917964
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[401]	training's rmse: 0.852093	valid_1's rmse: 0.917912
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[402]	training's rmse: 0.851812	valid_1's rmse: 0.917784
[403]	training's rmse: 0.851578	valid_1's rmse: 0.917648
[404]	training's rmse: 0.851428	valid_1's rmse: 0.917609
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[405]	training's rmse: 0.851164	valid_1's rmse: 0.917401
[406]	training's rmse: 0.850963	valid_1's rmse: 0.917342
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[407]	training's rmse: 0.850812	valid_1's rmse: 0.91731
[408]	training's rmse: 0.850638	valid_1's rmse: 0.91724
[409]	training's rmse: 0.850465	valid_1's rmse: 0.917127


[490]	training's rmse: 0.838356	valid_1's rmse: 0.913079
[491]	training's rmse: 0.838194	valid_1's rmse: 0.912961
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[492]	training's rmse: 0.838079	valid_1's rmse: 0.912948
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[493]	training's rmse: 0.83795	valid_1's rmse: 0.912921
[494]	training's rmse: 0.837822	valid_1's rmse: 0.912883
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[495]	training's rmse: 0.837712	valid_1's rmse: 0.912841
[496]	training's rmse: 0.837611	valid_1's rmse: 0.912818
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[497]	training's rmse: 0.837387	valid_1's rmse: 0.91267
[498]	training's rmse: 0.837273	valid_1's rmse: 0.912657
[499]	training's rmse: 0.837141	valid_1's rmse: 0.912699
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[500]	training's rmse: 0.837061	valid_1's rmse: 0.912685


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1686]	training's rmse: 0.776129	valid_1's rmse: 0.901428
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1687]	training's rmse: 0.776103	valid_1's rmse: 0.901419
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1688]	training's rmse: 0.776078	valid_1's rmse: 0.901422
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1689]	training's rmse: 0.776062	valid_1's rmse: 0.901416
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1690]	training's rmse: 0.776042	valid_1's rmse: 0.901429
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1691]	training's rmse: 0.776025	valid_1's rmse: 0.90143
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1692]	training's rmse: 0.776004	valid_1's rmse: 0.901433
[LightGBM] [Warning] No further splits with positive gain, best gain: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1750]	training's rmse: 0.774875	valid_1's rmse: 0.90152
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1751]	training's rmse: 0.77486	valid_1's rmse: 0.901521
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1752]	training's rmse: 0.774844	valid_1's rmse: 0.901525
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1753]	training's rmse: 0.774829	valid_1's rmse: 0.90151
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1754]	training's rmse: 0.774809	valid_1's rmse: 0.901497
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1755]	training's rmse: 0.774784	valid_1's rmse: 0.901501
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1756]	training's rmse: 0.774772	valid_1's rmse: 0.901501
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1813]	training's rmse: 0.773646	valid_1's rmse: 0.901505
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1814]	training's rmse: 0.773636	valid_1's rmse: 0.901506
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1815]	training's rmse: 0.773602	valid_1's rmse: 0.901517
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1816]	training's rmse: 0.773582	valid_1's rmse: 0.901514
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1817]	training's rmse: 0.773566	valid_1's rmse: 0.901516
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1818]	training's rmse: 0.773548	valid_1's rmse: 0.901523
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1819]	training's rmse: 0.773525	valid_1's rmse: 0.901532
[LightGBM] [Warning] No further splits with positive gain, best gain:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1876]	training's rmse: 0.77246	valid_1's rmse: 0.90138
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1877]	training's rmse: 0.772432	valid_1's rmse: 0.901382
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1878]	training's rmse: 0.772423	valid_1's rmse: 0.901382
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1879]	training's rmse: 0.77241	valid_1's rmse: 0.901379
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1880]	training's rmse: 0.772392	valid_1's rmse: 0.901406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1881]	training's rmse: 0.77237	valid_1's rmse: 0.901405
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1882]	training's rmse: 0.772347	valid_1's rmse: 0.901406
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1939]	training's rmse: 0.771343	valid_1's rmse: 0.901456
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1940]	training's rmse: 0.771326	valid_1's rmse: 0.901461
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1941]	training's rmse: 0.771305	valid_1's rmse: 0.901497
[1942]	training's rmse: 0.771278	valid_1's rmse: 0.901498
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1943]	training's rmse: 0.771266	valid_1's rmse: 0.901496
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1944]	training's rmse: 0.771247	valid_1's rmse: 0.901522
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1945]	training's rmse: 0.771222	valid_1's rmse: 0.90153
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1946]	training's rmse: 0.771193	valid_1's rmse: 0.901514
[LightGBM] [

In [23]:
# test数据
testData = matrix[matrix['date_block_num'] == 34]
X_test = testData.drop('item_cnt_month', axis=1)

In [24]:
# 预测&生成文件
y_test = lgb_model.predict(X_test).clip(0, 20)
submission = pd.DataFrame({ 'ID': range(0, 214200), 'item_cnt_month': y_test})

test0 = test[test.item_id.isin(six_zero_item_id)]
ids = list(test0.ID.values)
submission.loc[submission.ID.isin(ids), 'item_cnt_month'] = 0.0
submission.to_csv('./submit/sub1.csv', index=False)